In [1]:
import pandas as pd
import os
import logging
import warnings
import plotly.express as px
from matplotlib.colors import LinearSegmentedColormap

In [2]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("processamento.log"),
        logging.StreamHandler()
    ]
)

In [3]:
# Ignorar warnings do tipo FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
logging.info("Warnings do tipo FutureWarning ignorados.")

2025-07-30 16:59:22,977 - INFO - Warnings do tipo FutureWarning ignorados.


In [4]:
try:
    # Caminho para o primeiro arquivo (dados principais)
    base_dir = os.path.join("..", "excel", "out")
    file_name = "CDX_US_HY_spread_simple_analysis.xlsx"
    file_path = os.path.join(base_dir, file_name)
    logging.info(f"Lendo arquivo principal: {file_path}")
    regression = pd.read_excel(file_path)
    logging.info("Arquivo principal carregado com sucesso.")

    # Caminho para o segundo arquivo (legendas)
    base_dir_l = os.path.join("..", "excel", "in")
    file_name_l = "legendas.xlsx"
    file_path_l = os.path.join(base_dir_l, file_name_l)
    logging.info(f"Lendo arquivo de legendas: {file_path_l}")
    leg = pd.read_excel(file_path_l)
    logging.info("Arquivo de legendas carregado com sucesso.")

    # Renomear e adicionar coluna
    regression.columns.values[0] = "Codes"
    regression.insert(1, "Names", "")
    logging.info("Coluna 'Codes' renomeada e 'Names' inserida.")

    # Criar mapeamento e exibir
    mapping = dict(zip(leg["Codes"], leg["Names"]))
    logging.info(f"Mapa de legendas criado com {len(mapping)} entradas.")
    logging.debug(f"Mapping: {mapping}")  # Use DEBUG para não poluir o terminal

    # Preencher nomes com base nos códigos
    preenchidos = 0
    for idx, code in regression["Codes"].items():
        if code in mapping:
            regression.at[idx, "Names"] = mapping[code]
            preenchidos += 1

    logging.info(f"{preenchidos} códigos preenchidos com nomes correspondentes.")

except FileNotFoundError as e:
    logging.error(f"Arquivo não encontrado: {e}")
    raise
except Exception as e:
    logging.error(f"Erro durante o processamento: {e}")
    raise


2025-07-30 16:59:22,994 - INFO - Lendo arquivo principal: ..\excel\out\CDX_US_HY_spread_simple_analysis.xlsx
2025-07-30 16:59:23,217 - INFO - Arquivo principal carregado com sucesso.
2025-07-30 16:59:23,218 - INFO - Lendo arquivo de legendas: ..\excel\in\legendas.xlsx
2025-07-30 16:59:23,233 - INFO - Arquivo de legendas carregado com sucesso.
2025-07-30 16:59:23,234 - INFO - Coluna 'Codes' renomeada e 'Names' inserida.
2025-07-30 16:59:23,235 - INFO - Mapa de legendas criado com 255 entradas.
2025-07-30 16:59:23,237 - INFO - 189 códigos preenchidos com nomes correspondentes.


In [5]:
cmap_coef = LinearSegmentedColormap.from_list(
    "coef_cmap",
    ["#F8696B", "#FFEB84", "#63BE7B"]
)

In [6]:
# Função para destacar p-values < 0.05
def highlight_pval(v):
    return "color: red" if v < 0.05 else ""

In [7]:
regression_styled = (
    regression.style
      # 3-color nas colunas mean e std (usando um colormap padrão)
      .background_gradient(
          subset=["mean", "std"],
          #cmap="viridis"
          cmap=cmap_coef
      )
      # 3-color em R2
      .background_gradient(
          subset=["R2"],
          #cmap="Blues"
          cmap=cmap_coef
      )
      # 3-color custom em coef
      .background_gradient(
          subset=["coef"],
          cmap=cmap_coef
      )
      # fonte vermelha em p-values abaixo de 0.05
      .applymap(
          highlight_pval,
          subset=["pval_ADR", "pval_Coint", "pval"]
      )
      #.set_caption("Regression")
)

### QUAIS VARIAVEIS EXPLICAM BEM O COMPORTAMENTO DO CDX_US_HX_SPREAD AO LONGO DO TEMPO ?

In [8]:
#regression_styled

### Quanto da variação de Y é explicada pelas variáveis X do meu modelo?

In [9]:
# Filtrar Top 20 e remover valores inválidos
top_n = 20
df_top_20_regression = regression.sort_values(by='R2', ascending=False).dropna(subset=['Names']).head(top_n)

label_map = {
    "CDX HY BBG": "CDX HY",
    "JPM FX Vol": "JPM FX Vol",
    "VIX SPX": "VIX SPX",
    "ISM Service (ZS)": "ISM Srv",
    "CDX IG BBG": "CDX IG",
    "VIX Eurostoxx": "VIX Eurostoxx",
    "Global PMI Manufacturing (ZS)": "Gbl Mfg",
    "Eurozone Manufacturing PMI SA (ZS)": "EZ Mfg",
    "Ted Spread Europe (%)": "TED Spread",
    "MOVE": "MOVE",
    "ISM Manufacturing (ZS)": "ISM Mfg",
    "10y Real Note Germany (%)": "10y Germany",
    "Global PMI Composite (ZS)": "Gbl Comp",
    "US Retail Sales SA (6m %ave chg)": "US Retail 6m Avg %",
    "Eurozone Composite PMI SA (ZS)": "EZ Comp",
    "Global PMI Service (ZS)": "Gbl Srv",
    "US Industrial Production SA (6m %ave chg)": "US Ind Prod 6m Avg %",
    "GDP US Chained 2012 Dollars YoY SA (%)": "GDP US YoY",
    "Fed SPF Residential Investment 1y (Dispersion P75 - P25) (%)": "SPF Invest Dispersion",
    "Eurostat Industrial Production Eurozone SWDA (6m %ave chg)": "EZ Ind Prod 6m Avg %",
}

df_top_20_regression['Label'] = df_top_20_regression['Names'].map(label_map)

fig = px.bar(
    df_top_20_regression,
    x='R2',
    y='Label',
    orientation='h',
    hover_name='Names',
    text='R2',
    template='plotly_white',
    title='Top 20 séries por R² (com destaque de cor)',
    color='R2',  # define a intensidade de cor com base no R²
    color_continuous_scale=[(0, "#84c4e9"), (1, "#104eab")],  # você pode testar 'Viridis', 'Cividis', 'Plasma', etc.
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

# Layout refinado
fig.update_layout(
    yaxis=dict(autorange="reversed"),
    xaxis_range=[0, 1],
    margin=dict(t=60, b=40, l=180, r=20),
    coloraxis_showscale=False  # opcional: esconde a barra de escala lateral
)

In [ ]:
#### PARTE 2 - Processamento de regs_summary - GERADO POR CDX_US_HY_spread_model.py

try:
    # Caminho para o primeiro arquivo (dados principais)
    base_dir = os.path.join("..", "excel", "out", 'US', 'Credit', 'HY', 't_0') 
    file_name = "sectors_regs_summary.xlsx"
    file_path = os.path.join(base_dir, file_name)
    logging.info(f"Lendo arquivo regs_summary: {file_path}")
    summary = pd.read_excel(file_path)
    logging.info("Arquivo regs_summary carregado com sucesso.")

    # Renomear e adicionar coluna
    summary.rename(columns={"idx": "Codes"}, inplace=True)
    summary.insert(summary.columns.get_loc("Codes") + 1, "Names", "")
    logging.info("Coluna 'Codes' renomeada e 'Names' inserida.")
    
    # adicionar coef e pval
    summary.insert(summary.columns.get_loc("Codes") + 1, "coef", summary['Codes'].str.endswith('(coef)'))
    summary.insert(summary.columns.get_loc("Codes") + 2, "pval", summary['Codes'].str.endswith('(pval)'))
    
    # Preenche coef e pval com True/False
    for idx, i in summary['coef'].items():
        if i == True:
            summary.at[idx, 'Codes'] = summary.at[idx, 'Codes'].replace(" (coef)", "")
            

    for idx, i in summary['pval'].items():
        if i == True:
            summary.at[idx, 'Codes'] = summary.at[idx - 1, 'Codes']
    
    # Criar mapeamento e exibir
    mapping = dict(zip(leg["Codes"], leg["Names"]))
    logging.info(f"Mapa de legendas criado com {len(mapping)} entradas.")
    logging.debug(f"Mapping: {mapping}") 

    # Preencher nomes com base nos códigos
    preenchidos = 0
    for idx, code in summary["Codes"].items():
        if code in mapping:
            summary.at[idx, "Names"] = mapping[code]
            preenchidos += 1

    logging.info(f"{preenchidos} códigos preenchidos com nomes correspondentes.")

except FileNotFoundError as e:
    logging.error(f"Arquivo não encontrado: {e}")
    raise
except Exception as e:
    logging.error(f"Erro durante o processamento: {e}")
    raise


2025-07-30 16:59:23,831 - INFO - Lendo arquivo regs_summary: ..\excel\out\US\Credit\HY\t_0\sectors_regs_summary.xlsx
2025-07-30 16:59:23,846 - INFO - Arquivo regs_summary carregado com sucesso.
2025-07-30 16:59:23,849 - INFO - Coluna 'Codes' renomeada e 'Names' inserida.
2025-07-30 16:59:23,857 - INFO - Mapa de legendas criado com 255 entradas.
2025-07-30 16:59:23,860 - INFO - 12 códigos preenchidos com nomes correspondentes.


In [11]:
df_coef = summary[
    (summary['coef'] == True) |
    ((summary['pval'] == False) & (summary['coef'] == False))
]

df_pval = summary[
    (summary['pval'] == True) |
    ((summary['pval'] == False) & (summary['coef'] == False))
]

In [12]:
last_7_codes = df_coef['Codes'].iloc[-7:]
df_coef.loc[df_coef.index[-7:], 'Names'] = last_7_codes.values
df_pval.loc[df_pval.index[-7:], 'Names'] = last_7_codes.values

In [13]:
coef = df_coef.copy()
pval = df_pval.copy()
coef.drop(columns=['Unnamed: 0','Codes', 'coef', 'pval'], inplace=True)
pval.drop(columns=['Unnamed: 0','Codes', 'coef', 'pval'], inplace=True)

In [14]:
# Criar dois DataFrames a partir do df_coef
df_main = coef.iloc[:-7].copy()  # Todos menos as 7 últimas linhas
df_statistics = coef.iloc[-7:].copy()  # As 7 últimas linhas (dados da regressão como AIC, BIC, DW, R²)
df_statistics = df_statistics.drop(columns=["Signal"], errors="ignore")

cols_to_convert_main = df_main.columns.difference(['Signal', 'Names'])
df_main[cols_to_convert_main] = df_main[cols_to_convert_main].apply(pd.to_numeric, errors='coerce').round(3)

cols_to_convert_stats = df_statistics.columns.difference(['Names'])
df_statistics[cols_to_convert_stats] = df_statistics[cols_to_convert_stats].apply(pd.to_numeric, errors='coerce').round(3)


df_statistics['best'] = df_statistics.iloc[:, 1:].max(axis=1)
df_main = df_main.fillna('')
df_statistics = df_statistics.fillna('')

display(df_main)
display(df_statistics)


Names CDX_US_HY_spread  \
0   Fed Sloos Tight Std for C&I Loans (Small Firms...                    
2                           Global PMI Composite (ZS)           -0.825   
4                                          JPM FX Vol                    
6                                                MOVE                    
8                                   Ted Spread US (%)            1.579   
10                                            VIX SPX                    

   CDX_US_HY_spread1 CDX_US_HY_spread2 CDX_US_HY_spread3 CDX_US_HY_spread4  \
0              0.718              0.44                                       
2                               -0.605                                       
4                                                  5.885             4.331   
6                                                                            
8              1.602             1.228             0.695             0.339   
10                                                                   0.787   

   CDX_US_HY_spread5 CDX_US_HY_spread6 CDX_US_HY_spread7 CDX_US_HY_spread8  \
0                                                                            
2                               -0.384            -0.502            -0.517   
4              4.465             4.189             4.598             4.648   
6                                0.011             0.016             0.017   
8                                                  0.168                     
10             0.837             0.424                                       

   CDX_US_HY_spread9 CDX_US_HY_spread10 CDX_US_HY_spread11 CDX_US_HY_spread12  \
0                                                                               
2             -0.374             -0.342             -0.384             -0.384   
4              4.158              4.452              4.189              4.189   
6               0.01                                 0.011              0.011   
8              0.123              0.187                                         
10             0.418              0.492              0.424              0.424   

   CDX_US_HY_spread13 CDX_US_HY_spread14 CDX_US_HY_spread15  
0                                                            
2              -0.384             -0.384             -0.384  
4               4.189              4.189              4.189  
6               0.011              0.011              0.011  
8                                                            
10              0.424              0.424              0.424

Names  CDX_US_HY_spread  CDX_US_HY_spread1  CDX_US_HY_spread2  \
12  intercept             0.037              0.038              0.039   
14        AIC         -1499.660          -1484.170          -1519.980   
15        BIC         -1488.940          -1473.450          -1505.670   
16         DW             0.200              0.190              0.200   
17       LMpv             0.000              0.000              0.000   
18        MSE             0.000              0.000              0.000   
19         R2             0.450              0.420              0.500   

    CDX_US_HY_spread3  CDX_US_HY_spread4  CDX_US_HY_spread5  \
12             -0.014             -0.014             -0.014   
14          -1678.920          -1732.180          -1730.540   
15          -1668.190          -1717.880          -1719.810   
16              0.370              0.460              0.440   
17              0.000              0.000              0.000   
18              0.000              0.000              0.000   
19              0.720              0.770              0.770   

    CDX_US_HY_spread6  CDX_US_HY_spread7  CDX_US_HY_spread8  \
12             -0.011             -0.011             -0.011   
14          -1761.070          -1748.690          -1749.770   
15          -1743.190          -1730.810          -1735.470   
16              0.430              0.420              0.410   
17              0.000              0.000              0.000   
18              0.000              0.000              0.000   
19              0.800              0.790              0.790   

    CDX_US_HY_spread9  CDX_US_HY_spread10  CDX_US_HY_spread11  \
12             -0.011              -0.009              -0.011   
14          -1759.580           -1757.240           -1761.070   
15          -1738.130           -1739.360           -1743.190   
16              0.430               0.440               0.430   
17              0.000               0.000               0.000   
18              0.000               0.000               0.000   
19              0.800               0.800               0.800   

    CDX_US_HY_spread12  CDX_US_HY_spread13  CDX_US_HY_spread14  \
12              -0.011              -0.011              -0.011   
14           -1761.070           -1761.070           -1761.070   
15           -1743.190           -1743.190           -1743.190   
16               0.430               0.430               0.430   
17               0.000               0.000               0.000   
18               0.000               0.000               0.000   
19               0.800               0.800               0.800   

    CDX_US_HY_spread15      best  
12              -0.011     0.039  
14           -1761.070 -1484.170  
15           -1743.190 -1473.450  
16               0.430     0.460  
17               0.000     0.000  
18               0.000     0.000  
19               0.800     0.800

In [15]:
def style_df_main(df):
    def color_gradient(val):
        try:
            if pd.isna(val):
                return ''
            elif val > 0:
                return f'background-color: rgba(0, 128, 0, {min(val / 5, 1)})'
            else:
                return f'background-color: rgba(255, 0, 0, {min(abs(val) / 5, 1)})'
        except:
            return ''

    styled = df.style
    styled = styled.applymap(color_gradient, subset=df.columns.difference(['Signal', 'Names']))
    return styled

In [16]:
def style_df_stats(df):
    def style_row(row):
        name = df.loc[row.name, 'Names'] 
        styled = [''] * len(row)
        if name == 'R2':
            styled = [''] + ['background-color: #d0f0c0' if v >= 0.75 else 'background-color: #fdd' for v in row[1:]]
        elif name == 'DW':
            styled = [''] + ['background-color: #fdd' if (v < 1.5 or v > 2.5) else '' for v in row[1:]]
        elif name == 'LMpv':
            styled = [''] + ['background-color: #fdd' if v < 0.05 else '' for v in row[1:]]
        elif name in ['AIC', 'BIC']:
            min_val = row[1:].min()
            styled = [''] + ['background-color: #cce5ff' if v == min_val else '' for v in row[1:]]
        
        return styled

    return df.style.apply(style_row, axis=1)


O beta das regressões refere-se aos coeficientes de regressão (`coef`) que representam o impacto de cada variável independente na variável dependente. Esses valores estão presentes na coluna `coef` do DataFrame `regression` ou `df_top_20_regression`, dependendo do contexto.

Por exemplo, no DataFrame `df_top_20_regression`, os valores de beta (coeficientes) para as variáveis estão na coluna `coef`. Esses coeficientes indicam a magnitude e a direção do impacto de cada variável explicativa no comportamento do CDX_US_HY_spread.

In [17]:
# Exibir os DataFrames com formatação
display(style_df_main(df_main))

Dados Estatisticos:

In [19]:
display(style_df_stats(df_statistics))

# TERCEIRA TABELA


In [ ]:
import pandas as pd

# Read the Excel file
file_path = '../excel/out/US/Credit/HY/t_0/reccursive/sectors_regs_summary.xlsx'
data = pd.read_excel(file_path)

leg = pd.read_excel("../excel/in/legendas.xlsx")
# Display the first few rows of the dataframe
# display(data.head())

In [ ]:
data.rename(columns={"idx": "Codes"}, inplace=True)
data.insert(data.columns.get_loc("Codes") + 1, "Names", "")
# display(data)

In [ ]:
data.insert(data.columns.get_loc("Codes") + 1, "coef", data['Codes'].str.endswith('(coef)'))
data.insert(data.columns.get_loc("Codes") + 2, "pval", data['Codes'].str.endswith('(pval)'))

In [ ]:
for idx, i in data['coef'].items():
    if i == True:
        data.at[idx, 'Codes'] = data.at[idx, 'Codes'].replace(" (coef)", "")
        

for idx, i in data['pval'].items():
    if i == True:
        data.at[idx, 'Codes'] = data.at[idx - 1, 'Codes']

In [ ]:
mapping = dict(zip(leg["Codes"], leg["Names"]))
# print(mapping)

for idx, code in data["Codes"].items():
    if code in mapping:
        data.at[idx, "Names"] = mapping[code]

In [ ]:
df_completo = data

In [ ]:
df_coef = data[
    (data['coef'] == True) |
    ((data['pval'] == False) & (data['coef'] == False))
]

df_pval = data[
    (data['pval'] == True) |
    ((data['pval'] == False) & (data['coef'] == False))
]

# display(df_coef)
# display(df_pval)

In [ ]:
# Seleciona as últimas 7 linhas de 'Codes' e atribui o mesmo valor a 'Names'
last_7_codes = df_coef['Codes'].iloc[-7:]
df_coef.loc[df_coef.index[-7:], 'Names'] = last_7_codes.values
df_pval.loc[df_pval.index[-7:], 'Names'] = last_7_codes.values


In [ ]:
# display(df_coef)
# display(df_pval)

In [ ]:
df_coef.copy()
df_pval.copy()
df_coef.drop(columns=['Unnamed: 0','Codes', 'coef', 'pval'], inplace=True)
df_pval.drop(columns=['Unnamed: 0','Codes', 'coef', 'pval'], inplace=True)

C:\Users\gabriel\AppData\Local\Temp\ipykernel_13764\1705058012.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gabriel\AppData\Local\Temp\ipykernel_13764\1705058012.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# display(df_coef)
# display(df_pval)

In [ ]:
df_coef.insert(df_coef.columns.get_loc("Names"), "Signal", "")
# df_coef

In [ ]:
# df_coef.columns 

In [ ]:
# Renomear as colunas, mantendo "Signal" e "Names" intactas
new_columns = []

for col in df_coef.columns:
    if col.startswith("CDX_US_HY_spread"):
        # Extraia o número do final (ou 0 se não houver sufixo)
        suffix = col.replace("CDX_US_HY_spread", "")
        number = int(suffix) + 1 if suffix.isdigit() else 1
        new_columns.append(str(number))
    else:
        new_columns.append(col)

# Atribuir os novos nomes ao DataFrame
df_coef.columns = new_columns


In [ ]:
new_columns = []

for col in df_pval.columns:
    if col.startswith("CDX_US_HY_spread"):
        # Extraia o número do final (ou 0 se não houver sufixo)
        suffix = col.replace("CDX_US_HY_spread", "")
        number = int(suffix) + 1 if suffix.isdigit() else 1
        new_columns.append(str(number))
    else:
        new_columns.append(col)

# Atribuir os novos nomes ao DataFrame
df_pval.columns = new_columns


In [ ]:
output_path = '../excel/out/df_coef_exported.xlsx'
df_coef.to_excel(output_path, index=False)

output_path2 = '../excel/out/df_pval_exported.xlsx'
df_pval.to_excel(output_path2, index=False)

In [ ]:
df_completo = df_completo[df_completo['coef'] == True]
# df_completo

In [ ]:
# Configura o logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s — %(levelname)s — %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

In [ ]:
notebook_dir = os.getcwd()
file_path = os.path.abspath(
    os.path.join(notebook_dir, '..', 'excel', 'in', 'out_DB_D.xlsx')
)
try:
    df = pd.read_excel(file_path)
    logger.info(f"Arquivo lido com sucesso: {file_path}")
except Exception as e:
    logger.error(f"Falha ao ler o arquivo {file_path}: {e}")
    raise

2025-07-30 16:45:01,205 - INFO - Arquivo lido com sucesso: d:\assistente_pesquisa\pietcon.github.io\regressoes\cdx_us_hy\excel\in\out_DB_D.xlsx


In [ ]:
# df.head()

In [ ]:
#Pegar as colunas
cols = df.columns.tolist()
# cols

In [ ]:
comuns = list(set(df_completo['Codes'].tolist()) & set(cols))
comuns.insert(0, "date")
# print(comuns)

In [ ]:
df_filtered_serie = df.loc[df.index[-30:], comuns]
pd.set_option('display.max_rows', None)  # Mostra todas as linhas
pd.set_option('display.max_columns', None)  # Mostra todas as colunas
# Conta o número de linhas e colunas no dataframe
num_linhas, num_colunas = df_filtered_serie.shape

# display(df_filtered_serie)

# print(f"Número de linhas: {num_linhas}")
# print(f"Número de colunas: {num_colunas}")

In [ ]:
delta_x = df_filtered_serie.iloc[-1] - df_filtered_serie.iloc[0]
# print(delta_x)

In [ ]:
filtered_codes = df_completo[df_completo['Codes'].isin(comuns)]
filtered_codes = filtered_codes[['Codes', 'Names']]
# display(filtered_codes)


In [ ]:
# Cria dicionários para início, fim e delta
inicios_dict = {}
fins_dict = {}
deltas_dict = {}

# Itera sobre as variáveis filtradas dinamicamente
for code in filtered_codes['Codes']:
    if code in df_filtered_serie.columns:
        inicio = df_filtered_serie.iloc[0][code]
        fim = df_filtered_serie.iloc[-1][code]
        delta = fim - inicio
    else:
        inicio = fim = delta = None  # Caso a variável não esteja no df_filtered_serie
    
    inicios_dict[code] = inicio
    fins_dict[code] = fim
    deltas_dict[code] = delta


In [ ]:
# inicios_dict

In [ ]:
# Adiciona colunas ao DataFrame com base nos dicionáriosA
filtered_codes['inicio'] = filtered_codes['Codes'].map(inicios_dict)
filtered_codes['fim'] = filtered_codes['Codes'].map(fins_dict)
filtered_codes['delta'] = filtered_codes['Codes'].map(deltas_dict)

In [ ]:
# display(filtered_codes)

In [ ]:
# display(df_completo)

In [ ]:
# Identifica colunas que começam com 'CDX_US_HY_spread'
coef_cols = [col for col in df_completo.columns if col.startswith('CDX_US_HY_spread')]

# Seleciona a linha do intercept
intercept_row = df_completo[df_completo['Codes'] == 'intercept']

# Cria o dicionário com chaves numéricas e valores do intercept
if not intercept_row.empty:
    intercept = {idx + 1: intercept_row.iloc[0][col] for idx, col in enumerate(coef_cols)}
else:
    intercept = {}

# print(intercept)


In [ ]:
coef_cols = [col for col in df_completo.columns if col.startswith('CDX_US_HY_spread')]
# print("Colunas que começam com 'CDX_US_HY_spread':", coef_cols)

for i in range(len(filtered_codes)):
    code = filtered_codes.iloc[i]['Codes']
    delta = deltas_dict.get(code, None)

    if delta is not None:

        linha_completa = df_completo[df_completo['Codes'] == code]

        if not linha_completa.empty:
            for idx, col in enumerate(coef_cols, start=1): 
                coef_value = linha_completa.iloc[0][col]
                if pd.notna(coef_value):
                    filtered_codes.loc[filtered_codes.index[i], f'{idx}'] = coef_value * delta


In [ ]:
# filtered_codes

In [ ]:
# Reorganiza o DataFrame
colunas_numericas = sorted([col for col in filtered_codes.columns if col.isdigit()], key=int)
colunas_fixas = [col for col in filtered_codes.columns if not col.isdigit()]
nova_ordem = colunas_fixas + colunas_numericas
filtered_codes = filtered_codes[nova_ordem]
filtered_codes

Codes              Names    inicio       fim     delta         1  \
4         jpm_vol         JPM FX Vol  0.007482  0.006648 -0.000834       NaN   
6            move               MOVE  0.722172  0.768208  0.046036       NaN   
8   ted_spread_us  Ted Spread US (%)  0.002230  0.001814 -0.000416 -0.000656   
10            vix            VIX SPX  0.013360  0.013893  0.000533       NaN   

           2        3         4         5         6         7         8  \
4        NaN      NaN -0.004908 -0.003613 -0.003724 -0.003494 -0.003835   
6        NaN      NaN       NaN       NaN       NaN  0.000497  0.000718   
8  -0.000666 -0.00051 -0.000289 -0.000141       NaN       NaN -0.000070   
10       NaN      NaN       NaN  0.000419  0.000446  0.000226       NaN   

           9        10        11        12        13        14        15  \
4  -0.003877 -0.003468 -0.003713 -0.003494 -0.003494 -0.003494 -0.003494   
6   0.000764  0.000470       NaN  0.000497  0.000497  0.000497  0.000497   
8        NaN -0.000051 -0.000078       NaN       NaN       NaN       NaN   
10       NaN  0.000223  0.000262  0.000226  0.000226  0.000226  0.000226   

          16  
4  -0.003494  
6   0.000497  
8        NaN  
10  0.000226

In [ ]:
df_filtered_serie

date  ted_spread_us   jpm_vol       vix      move
16095 2024-01-25       0.002230  0.007482  0.013360  0.722172
16096 2024-01-26       0.002198  0.007432  0.013173  0.694297
16097 2024-01-27       0.002198  0.007432  0.013173  0.694297
16098 2024-01-28       0.002198  0.007432  0.013173  0.694297
16099 2024-01-29       0.002130  0.007532  0.013508  0.724355
16100 2024-01-30       0.002078  0.007532  0.013222  0.714224
16101 2024-01-31       0.002087  0.007631  0.014248  0.728900
16102 2024-02-01       0.001671  0.007581  0.013785  0.760432
16103 2024-02-02       0.001799  0.007611  0.013755  0.726195
16104 2024-02-03       0.001799  0.007611  0.013755  0.726195
16105 2024-02-04       0.001799  0.007611  0.013755  0.726195
16106 2024-02-05       0.002035  0.007720  0.013577  0.735632
16107 2024-02-06       0.002080  0.007621  0.012975  0.744790
16108 2024-02-07       0.001937  0.007541  0.012748  0.731695
16109 2024-02-08       0.001768  0.007442  0.012709  0.735248
16110 2024-02-09       0.001875  0.007283  0.012847  0.723725
16111 2024-02-10       0.001875  0.007283  0.012847  0.723725
16112 2024-02-11       0.001875  0.007283  0.012847  0.723725
16113 2024-02-12       0.001760  0.007134  0.013834  0.740889
16114 2024-02-13       0.001754  0.007214  0.015726  0.757154
16115 2024-02-14       0.001951  0.007214  0.014278  0.751275
16116 2024-02-15       0.002005  0.007075  0.013913  0.751841
16117 2024-02-16       0.001964  0.006946  0.014140  0.738121
16118 2024-02-17       0.001964  0.006946  0.014140  0.738121
16119 2024-02-18       0.001964  0.006946  0.014140  0.738121
16120 2024-02-19       0.001964  0.006876  0.014603  0.738121
16121 2024-02-20       0.002161  0.006767  0.015302  0.746783
16122 2024-02-21       0.001854  0.006717  0.015224  0.762067
16123 2024-02-22       0.001784  0.006648  0.014435  0.768208
16124 2024-02-23       0.001814  0.006648  0.013893  0.768208

In [ ]:
print("Inícios:", inicios_dict)
print("Fins:", fins_dict)
print("Deltas:", deltas_dict)
print("Intercept", intercept)

Inícios: {'jpm_vol': np.float64(0.007481940347755529), 'move': np.float64(0.7221718596005616), 'ted_spread_us': np.float64(0.002229512788283484), 'vix': np.float64(0.01336035170183639)}
Fins: {'jpm_vol': np.float64(0.006647853971464475), 'move': np.float64(0.7682082697818243), 'ted_spread_us': np.float64(0.001813754153032583), 'vix': np.float64(0.01389304318742333)}
Deltas: {'jpm_vol': np.float64(-0.0008340863762910534), 'move': np.float64(0.0460364101812627), 'ted_spread_us': np.float64(-0.00041575863525090094), 'vix': np.float64(0.0005326914855869388)}
Intercept {1: 0.0374, 2: 0.0381, 3: 0.0387, 4: -0.0145, 5: -0.0136, 6: -0.0145, 7: -0.0112, 8: -0.0107, 9: -0.0111, 10: -0.0108, 11: -0.009, 12: -0.0112, 13: -0.0112, 14: -0.0112, 15: -0.0112, 16: -0.0112}


In [ ]:
coef_cols

['CDX_US_HY_spread',
 'CDX_US_HY_spread1',
 'CDX_US_HY_spread2',
 'CDX_US_HY_spread3',
 'CDX_US_HY_spread4',
 'CDX_US_HY_spread5',
 'CDX_US_HY_spread6',
 'CDX_US_HY_spread7',
 'CDX_US_HY_spread8',
 'CDX_US_HY_spread9',
 'CDX_US_HY_spread10',
 'CDX_US_HY_spread11',
 'CDX_US_HY_spread12',
 'CDX_US_HY_spread13',
 'CDX_US_HY_spread14',
 'CDX_US_HY_spread15']

In [ ]:
# Mapeia os coef_cols para índices 1, 2, 3, ... para bater com o dicionário de intercept
coef_col_index_map = {col: idx+1 for idx, col in enumerate(coef_cols)}

# Inicializa dicionário de previsões
initial_preds = {}

# Loop por cada série (code)
for code in inicios_dict:
    inicio_val = inicios_dict[code]
    row = df_completo[df_completo['Codes'] == code]

    if row.empty:
        continue

    pred = 0.0
    for col in coef_cols:
        beta = row.iloc[0][col]
        if pd.notna(beta):
            pred += beta * inicio_val

            # Adiciona intercepto correspondente ao índice
            intercept_idx = coef_col_index_map[col]
            pred += intercept.get(intercept_idx, 0)

    initial_preds[code] = pred

print("Initial Predictions:", initial_preds)


Initial Predictions: {'jpm_vol': np.float64(0.28006629139226163), 'move': np.float64(-0.02238317665081979), 'ted_spread_us': np.float64(0.06880205997582063), 'vix': np.float64(-0.04724544577671334)}


### Initial Prediction 
$$
\text{Initial Pred}_t = \sum_i \beta_{i,t} \cdot x_i(\text{inicio}) + \text{intercept}_t
$$

### Final Prediction
$$
\text{Final Pred}_t = \sum_i \beta_{i,t} \cdot x_i(\text{fim}) + \text{intercept}_t
$$

Aqui, você usa os `fins_dict` (valores finais) diretamente.

### Period Variation
$$
\text{Period Variation}_t = (\text{Final Pred}_t - \text{Initial Pred}_t) \times 100
$$

In [ ]:
# Inicializa dicionário das previsões finais recalculadas
final_preds = {}

# Loop por cada série (code)
for code in fins_dict:
    fim_val = fins_dict[code]
    row = df_completo[df_completo['Codes'] == code]

    if row.empty:
        continue

    pred = 0.0
    for col in coef_cols:
        beta = row.iloc[0][col]
        if pd.notna(beta):
            pred += beta * fim_val

            # Adiciona intercepto da regressão correspondente
            intercept_idx = coef_col_index_map[col]
            pred += intercept.get(intercept_idx, 0)

    final_preds[code] = pred

print("Final Predictions (full):", final_preds)


Final Predictions (full): {'jpm_vol': np.float64(0.23196644847022196), 'move': np.float64(-0.017448073479388433), 'ted_spread_us': np.float64(0.06634014521718244), 'vix': np.float64(-0.04454001225971439)}


In [ ]:
# Inicializa dicionário de variação
period_variation = {}

for code in initial_preds:
    final_val = final_preds.get(code)
    initial_val = initial_preds[code]

    if final_val is not None:
        variation = (final_val - initial_val) * 100  # em pontos-base
        period_variation[code] = variation

print("Period Variation (bps):", period_variation)

Period Variation (bps): {'jpm_vol': np.float64(-4.809984292203967), 'move': np.float64(0.49351031714313587), 'ted_spread_us': np.float64(-0.24619147586381895), 'vix': np.float64(0.2705433516998951)}


In [ ]:
df_result = pd.DataFrame([initial_preds, final_preds, period_variation],
                         index=['Initial Pred', 'FinalA Pred', 'Period Variation (bps)'])

### EXPOR DADOS

In [ ]:
filtered_codes_expor = filtered_codes.copy()


In [ ]:
import pandas as pd

# Suponha que df_result e filtered_codes_expor já estejam definidos

# Cria o dicionário de mapeamento a partir do filtered_codes_expor
code_name_map = dict(zip(filtered_codes_expor['Codes'], filtered_codes_expor['Names']))

# Renomeia as colunas do df_result com base no dicionário
df_result_renamed = df_result.rename(columns=code_name_map)

In [ ]:
filtered_codes_expor = filtered_codes_expor.fillna('')
filtered_codes_expor.drop(columns=['Codes'], inplace=True)

In [ ]:
# display(filtered_codes_expor)
# display(df_result_renamed)

In [ ]:
import pandas as pd

# Função para colorir o **fundo** da célula com base no valor
def background_delta(val):
    if pd.isna(val):
        return ''
    elif val > 0:
        return 'background-color: lightgreen'
    elif val < 0:
        return 'background-color: lightcoral'
    else:
        return 'background-color: lightgray'

# Aplica a formatação condicional no fundo da coluna "delta"
styled_df = filtered_codes_expor.style.applymap(background_delta, subset=['delta'])

# Exibe em notebook Jupyter (ou salva como HTML, se quiser)
styled_df


In [ ]:
import pandas as pd
import numpy as np

# Função de formatação horizontal com verificação de tipo numérico
def style_row(row):
    styles = []
    for col in row.index:
        val = row[col]
        if isinstance(val, (int, float, np.number)) and not pd.isna(val):
            if val > 0:
                styles.append('background-color: lightgreen')
            elif val < 0:
                styles.append('background-color: lightcoral')
            else:
                styles.append('background-color: lightgray')
        else:
            styles.append('')  # Deixa sem estilo se não for numérico
    return styles

# Define colunas para estilizar horizontalmente (da 'inicio' até '16')
columns_to_style = filtered_codes_expor.loc[:, '1':'16'].columns

# Aplica a formatação linha a linha nas colunas desejadas
styled_df = filtered_codes_expor.style.apply(style_row, axis=1, subset=columns_to_style)

# Exibir (em Jupyter) ou salvar como HTML
styled_df


In [ ]:
df_result_renamed

JPM FX Vol      MOVE  Ted Spread US (%)   VIX SPX
Initial Pred              0.280066 -0.022383           0.068802 -0.047245
FinalA Pred               0.231966 -0.017448           0.066340 -0.044540
Period Variation (bps)   -4.809984  0.493510          -0.246191  0.270543

In [ ]:
def highlight_variation(val):
    """Color negative in blue, positive in red."""
    color = ''
    if isinstance(val, (int, float)):
        if val > 0:
            color = 'color: red'
        elif val < 0:
            color = 'color: blue'
    return color

def add_arrows(val):
    """Add arrow emojis for direction."""
    if isinstance(val, (int, float)):
        if val > 0:
            return f"🔺 {val:.6f}"
        elif val < 0:
            return f"🔻 {val:.6f}"
    return f"{val:.6f}"

def bold_max(s):
    """Bold the most extreme value in a row (by magnitude)."""
    max_idx = np.abs(s).idxmax()
    return ['font-weight: bold' if idx == max_idx else '' for idx in s.index]

# Aplica os estilos
styled = df_result_renamed.copy()

# Aplica setas apenas na linha de variação
styled.loc['Period Variation (bps)'] = styled.loc['Period Variation (bps)'].apply(add_arrows)

# Renderizar com estilo
styled = df_result_renamed.style\
    .applymap(highlight_variation, subset=pd.IndexSlice['Period Variation (bps)', :])\
    .apply(bold_max, axis=1)\
    .format(precision=6)\
    .set_caption("Tabela com Formatação Condicional")

styled